# BBMs for the Optimal Power Flow

In this script, we aim to construct Black-Box Models (BBMs) (i.e., data-driven models) for the Optimal Power Flow (OPF) problem. We use the data stored in the 'data/OPF' folder, containing tables with the OPF IO data.

In general terms, the OPF problem is a non-linear mapping from the input space (i.e., the power injections) to the output space (i.e., the voltage magnitudes and angles): $$ \mathbf{y} = \mathbf{f}(\mathbf{E}, \mathbf{\Pi}), $$ where $\mathbf{E}$ is the vector of power injections and $\mathbf{\Pi}$ is the vector of active and reactive power demands, $\mathbf{\Pi}$ is a vector of the binary status of generators, transmission lines, and transformers (i.e., 1 "on" and 0 "off"), $\mathbf{f}(\cdot)$ is the OPF function, and $\mathbf{y}$ is the vector of optimal voltage magnitudes and angles.

In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sys
from importlib import reload

import greyboxmodels.bbmcpsmodels.creator as creator
import greyboxmodels.bbmcpsmodels.cyber.OPF as opf_bbm


# Set the working directory
os.chdir("D:/projects/IPTLC_BBMs/")
print(f"Working directory: {os.getcwd()}")

# Check GPU availability
device = creator.get_device()
print(f"Device: {device}")

Working directory: D:\projects\IPTLC_BBMs
Device: cuda:0


In [3]:
# Specify the paths
datasets_folder = Path("data/OPF/20240226_184851")

# # Print the available datasets
# print("Available datasets:")
# for path in datasets_folder.iterdir():
#     if path.is_dir() and "OPF" in path.name:
#         print(path.name)

# Setup the BBM creator

In [4]:
# Create the BBMCreator
reload(creator)
BBM_creator = creator.BBMCreator()

In [5]:
# Set up the dataset
dataset_name = "MinMaxNormalizedOPF"
loaders = creator.setup_datasets(datasets_folder,
                                 dataset_name,
                                 remove_nans=True,
                                 ratios=(0.70, 0.15, 0.15),
                                 batch_size=32,
                                 input_name="opf_inputs_minmax_normalized.npy",
                                 output_name="opf_outputs_minmax_normalized.npy")
BBM_creator.set_dataloaders(*loaders)

Loading: data\OPF\20240226_184851
---- Dataset loaded ----
Input shape: (8064, 51)
Output shape: (8064, 9)
---- Removing NaNs ----
Rows to delete: []
Input shape: (8064, 51)
Output shape: (8064, 9)
---- Converting to torch tensors ----
---- Dataset loaded! ----


# BBM 1: a two-layer feedforward neural network

In [5]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]

BBM_creator.instantiate_model(opf_bbm.BBM1_SimpleNet, input_size, output_size)

In [6]:
# Train the model
BBM_creator.train(save_to="models", epochs=100)

------ Training model 'BBM1_SimpleNet' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM1_SimpleNet_MinMaxNormalizedOPF_20240228-144804.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/100 (Loss - Train: 7.03e-03, Best val: N/A): 100%|██████████| 1109/1109 [00:07<00:00, 142.15it/s]
Epoch 2/100 (Loss - Train: 2.49e-03, Best val: 2.79e-03): 100%|██████████| 1109/1109 [00:04<00:00, 271.05it/s]
Epoch 3/100 (Loss - Train: 2.01e-03, Best val: 2.13e-03): 100%|██████████| 1109/1109 [00:04<00:00, 268.98it/s]
Epoch 4/100 (Loss - Train: 1.68e-03, Best val: 1.69e-03): 100%|██████████| 1109/1109 [00:04<00:00, 259.73it/s]
Epoch 5/100 (Loss - Train: 1.41e-03, Best val: 1.41e-03): 100%|██████████| 1109/1109 [00:04<00:00, 263.50it/s]
Epoch 6/100 (Loss - Train: 1.29e-03, Best val: 1.29e-03): 100%|██████████| 1109/1109 [00:04<00:00, 264.76it/s]
Epoch 7/100 (Loss - Train: 1.15e-03, Best val: 1.13e-03): 100%|██████████| 1109/1109 [00:04<00:00, 270.76it/s]
Epoch 8/100 (Loss - Train: 1.07e-03, Best val: 1.00e-03): 100%|██████████| 1109/1109 [00:04<00:00, 265.86it/s]
Epoch 9/100 (Loss - Train: 9.92e-04, Best val: 9.38e-04): 100%|██████████| 1109/1109 [00:04<00:00, 256.42it/s]
Epoch 

                                                    Train loss  \
Model          Dataset             Timestamp                     
BBM1_SimpleNet MinMaxNormalizedOPF 20240228-144804    0.000698   

                                                    Validation loss  \
Model          Dataset             Timestamp                          
BBM1_SimpleNet MinMaxNormalizedOPF 20240228-144804         0.000677   

                                                    Test loss  Input size  \
Model          Dataset             Timestamp                                
BBM1_SimpleNet MinMaxNormalizedOPF 20240228-144804   0.000667          51   

                                                    Output size  \
Model          Dataset             Timestamp                      
BBM1_SimpleNet MinMaxNormalizedOPF 20240228-144804            9   

                                                    Training time [ms]  \
Model          Dataset             Timestamp                             
BBM1_

D:\projects\IPTLC_BBMs\src\greyboxmodels\bbmcpsmodels\creator.py:413: PerformanceWarning: indexing past lexsort depth may impact performance.
  comparison = (model_name, dataset_name, timestamp) in results_table.index


In [7]:
BBM_creator._summary()

,,,Train loss,Validation loss,Test loss,Input size,Output size,Training time [ms],Model path
Model,Dataset,Timestamp,,,,,,,
BBM1_SimpleNet,MinMaxNormalizedOPF,20240228-144804,0.000694,0.000679,0.000671,51,9,464667.564,models\BBM1_SimpleNet_MinMaxNormalizedOPF_2024...


# BBM 2: a two-layer feedforward neural network

In [6]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]

BBM_creator.instantiate_model(opf_bbm.BBM2_DeepNN, input_size, output_size)

In [7]:
# Train the model
BBM_creator.train(save_to="models", epochs=100)

------ Training model 'BBM2-deep' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM2-deep_MinMaxNormalizedOPF_20240229-103342.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/100 (Loss - Train: 4.56e-02, Best val: N/A): 100%|██████████| 177/177 [00:03<00:00, 52.11it/s] 
Epoch 2/100 (Loss - Train: 1.34e-02, Best val: 0.02): 100%|██████████| 177/177 [00:00<00:00, 260.37it/s]
Epoch 3/100 (Loss - Train: 1.00e-02, Best val: 0.01): 100%|██████████| 177/177 [00:00<00:00, 226.59it/s]
Epoch 4/100 (Loss - Train: 7.00e-03, Best val: 7.57e-03): 100%|██████████| 177/177 [00:00<00:00, 226.88it/s]
Epoch 5/100 (Loss - Train: 5.28e-03, Best val: 5.70e-03): 100%|██████████| 177/177 [00:00<00:00, 258.41it/s]
Epoch 6/100 (Loss - Train: 3.81e-03, Best val: 3.84e-03): 100%|██████████| 177/177 [00:00<00:00, 245.81it/s]
Epoch 7/100 (Loss - Train: 3.39e-03, Best val: 3.36e-03): 100%|██████████| 177/177 [00:00<00:00, 219.87it/s]
Epoch 8/100 (Loss - Train: 3.07e-03, Best val: 3.20e-03): 100%|██████████| 177/177 [00:00<00:00, 234.43it/s]
Epoch 9/100 (Loss - Train: 3.20e-03, Best val: 2.97e-03): 100%|██████████| 177/177 [00:00<00:00, 252.86it/s]
Epoch 10/100 (Loss - Train: 2.94

                                               Train loss  Validation loss  \
Model     Dataset             Timestamp                                      
BBM2-deep MinMaxNormalizedOPF 20240229-103342    0.001148         0.001237   

                                               Test loss  Input size  \
Model     Dataset             Timestamp                                
BBM2-deep MinMaxNormalizedOPF 20240229-103342   0.001086          51   

                                               Output size  \
Model     Dataset             Timestamp                      
BBM2-deep MinMaxNormalizedOPF 20240229-103342            9   

                                               Training time [ms]  \
Model     Dataset             Timestamp                             
BBM2-deep MinMaxNormalizedOPF 20240229-103342           96384.134   

                                                                                      Model path  
Model     Dataset             Timestamp               

D:\projects\IPTLC_BBMs\src\greyboxmodels\bbmcpsmodels\creator.py:413: PerformanceWarning: indexing past lexsort depth may impact performance.
  comparison = (model_name, dataset_name, timestamp) in results_table.index


In [8]:
BBM_creator._summary()

,,,Train loss,Validation loss,Test loss,Input size,Output size,Training time [ms],Model path
Model,Dataset,Timestamp,,,,,,,
BBM2-deep,MinMaxNormalizedOPF,20240229-103342,0.001178,0.001285,0.001044,51,9,96384.134,models\BBM2-deep_MinMaxNormalizedOPF_20240229-...


# LEGACY CODE BELOW

# BBM 1: a two-layer feedforward neural network

## Normalized dataset

In [33]:
# Set up the dataset
dataset_name = "OPF-NI1-O3"
loaders = creator.setup_datasets(datasets_folder, dataset_name, remove_nans=True, ratios=(0.70, 0.15, 0.15), batch_size=32)
BBM_creator.set_dataloaders(*loaders)

# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]
BBM_creator.instantiate_model(opf_bbm.BBM1_SimpleNet, input_size, output_size)

Loading: data\OPF\2023-12-06_18-00-46\OPF-NI1-O3
---- Dataset loaded ----
Input shape: (59620, 53)
Output shape: (59620, 9)
---- Removing NaNs ----
Rows to delete: [57886 57887 57888 57889 57890 57891 57892 57893 57894 57895 57896 57897
 57898 57899 57900 57901 57902 57903 57904 57905 57906 57907 57908 57909
 57910 57911 57912 57913 57914 57915 57916 57917 57918 57919 57920 57921
 57922 57923 57924 57925 57926 57927 57928 57929 57930]
Input shape: (59575, 53)
Output shape: (59575, 9)
---- Converting to torch tensors ----
---- Dataset loaded! ----


In [34]:
# Train the model
BBM_creator.train(save_to="models", epochs=100)

------ Training model 'BBM1_SimpleNet' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM1_SimpleNet_OPF-NI1-O3_20240201-170922.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/100 (Loss - Train: 2.44e+02, Best val: N/A): 100%|██████████| 1304/1304 [00:04<00:00, 314.37it/s]
Epoch 2/100 (Loss - Train: 3.94e+01, Best val: 44.63): 100%|██████████| 1304/1304 [00:04<00:00, 317.37it/s]
Epoch 3/100 (Loss - Train: 3.79e+01, Best val: 43.36): 100%|██████████| 1304/1304 [00:04<00:00, 316.83it/s]
Epoch 4/100 (Loss - Train: 3.58e+01, Best val: 41.88): 100%|██████████| 1304/1304 [00:04<00:00, 317.26it/s]
Epoch 5/100 (Loss - Train: 3.32e+01, Best val: 38.12): 100%|██████████| 1304/1304 [00:04<00:00, 312.62it/s]
Epoch 6/100 (Loss - Train: 3.06e+01, Best val: 34.91): 100%|██████████| 1304/1304 [00:04<00:00, 314.01it/s]
Epoch 7/100 (Loss - Train: 2.81e+01, Best val: 30.88): 100%|██████████| 1304/1304 [00:04<00:00, 318.16it/s]
Epoch 8/100 (Loss - Train: 2.64e+01, Best val: 27.82): 100%|██████████| 1304/1304 [00:04<00:00, 319.93it/s]
Epoch 9/100 (Loss - Train: 2.53e+01, Best val: 26.34): 100%|██████████| 1304/1304 [00:04<00:00, 321.83it/s]
Epoch 10/100 (Loss - Train: 2.

KeyboardInterrupt: 

In [22]:
BBM_creator._summary()

,,,Train loss,Validation loss,Test loss,Input size,Output size,Training time [ms],Model path
Model,Dataset,Timestamp,,,,,,,
BBM1_SimpleNet,OPF-NI1-O3,20240201-162924,31.81697,31.556109,32.631261,53,9,490856.2,models\BBM1_SimpleNet_OPF-NI1-O3_20240201-1629...


## Raw dataset

In [59]:
# Set up the dataset
dataset_name = "OPF-I1-O3"
loaders = creator.setup_datasets(datasets_folder, dataset_name, remove_nans=True, ratios=(0.70, 0.15, 0.15), batch_size=32)
BBM_creator.set_dataloaders(*loaders)

Loading: data\OPF\2023-12-06_18-00-46\OPF-I1-O3
---- Dataset loaded ----
Input shape: (59620, 53)
Output shape: (59620, 9)
---- Removing NaNs ----
Rows to delete: [57886 57887 57888 57889 57890 57891 57892 57893 57894 57895 57896 57897
 57898 57899 57900 57901 57902 57903 57904 57905 57906 57907 57908 57909
 57910 57911 57912 57913 57914 57915 57916 57917 57918 57919 57920 57921
 57922 57923 57924 57925 57926 57927 57928 57929 57930]
Input shape: (59575, 53)
Output shape: (59575, 9)
---- Converting to torch tensors ----
---- Dataset loaded! ----


In [60]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]
BBM_creator.instantiate_model(opf_bbm.BBM1_SimpleNet, input_size, output_size)

In [61]:
# Train the model
BBM_creator.train(save_to="models", epochs=100)

------ Training model 'BBM1-2layers' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM1-2layers_OPF-I1-O3_20240201-111317.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/100 (Loss - Running: 108364.42, Avg. batch: 8.34e+01, Best avg.: N/A): 100%|██████████| 1304/1304 [00:03<00:00, 355.73it/s]
Epoch 2/100 (Loss - Running: 42921.78, Avg. batch: 3.30e+01, Best avg.: 40.91): 100%|██████████| 1304/1304 [00:03<00:00, 363.00it/s]
Epoch 3/100 (Loss - Running: 42141.25, Avg. batch: 3.24e+01, Best avg.: 37.92): 100%|██████████| 1304/1304 [00:03<00:00, 363.23it/s]
Epoch 4/100 (Loss - Running: 40906.66, Avg. batch: 3.15e+01, Best avg.: 37.42): 100%|██████████| 1304/1304 [00:04<00:00, 318.98it/s]
Epoch 5/100 (Loss - Running: 37906.08, Avg. batch: 2.92e+01, Best avg.: 36.41): 100%|██████████| 1304/1304 [00:03<00:00, 327.87it/s]
Epoch 6/100 (Loss - Running: 33244.90, Avg. batch: 2.56e+01, Best avg.: 31.29): 100%|██████████| 1304/1304 [00:03<00:00, 328.37it/s]
Epoch 7/100 (Loss - Running: 28678.70, Avg. batch: 2.21e+01, Best avg.: 25.16): 100%|██████████| 1304/1304 [00:04<00:00, 318.42it/s]
Epoch 8/100 (Loss - Running: 26155.95, Avg. batch: 2.01e+01, Best avg.

                                        Train loss  Validation loss  \
Model        Dataset   Timestamp                                      
BBM1-2layers OPF-I1-O3 20240201-111317   19.041072        18.338107   

                                        Test loss  Input size  Output size  \
Model        Dataset   Timestamp                                             
BBM1-2layers OPF-I1-O3 20240201-111317  18.260585          53            9   

                                        Training time [ms]  \
Model        Dataset   Timestamp                             
BBM1-2layers OPF-I1-O3 20240201-111317          422910.943   

                                                                              Model path  
Model        Dataset   Timestamp                                                          
BBM1-2layers OPF-I1-O3 20240201-111317  models\BBM1-2layers_OPF-I1-O3_20240201-111317.pt  
------ Finished! ------


D:\projects\IPTLC_BBMs\src\creator.py:405: PerformanceWarning: indexing past lexsort depth may impact performance.
  if comparison:


In [62]:
BBM_creator._summary()

,,,Train loss,Validation loss,Test loss,Input size,Output size,Training time [ms],Model path
Model,Dataset,Timestamp,,,,,,,
BBM1-2layers,OPF-I1-O3,20240201-111317,19.041072,18.338107,18.260585,53,9,422910.943,models\BBM1-2layers_OPF-I1-O3_20240201-111317.pt


# BBM 2: a two-layer feedforward neural network

## Normalized dataset

In [ ]:
# Assign the model
BBM2 = opf_bbm.BBM2_DeepNN

In [21]:
# Set up the dataset
dataset_name = "OPF-NI1-O3"
loaders = creator.setup_datasets(datasets_folder, dataset_name, remove_nans=True, ratios=(0.70, 0.15, 0.15), batch_size=32)
BBM_creator.set_dataloaders(*loaders)

Loading: data\OPF\2023-12-06_18-00-46\OPF-NI1-O3
---- Dataset loaded ----
Input shape: (59620, 53)
Output shape: (59620, 9)
---- Removing NaNs ----
Rows to delete: [57886 57887 57888 57889 57890 57891 57892 57893 57894 57895 57896 57897
 57898 57899 57900 57901 57902 57903 57904 57905 57906 57907 57908 57909
 57910 57911 57912 57913 57914 57915 57916 57917 57918 57919 57920 57921
 57922 57923 57924 57925 57926 57927 57928 57929 57930]
Input shape: (59575, 53)
Output shape: (59575, 9)
---- Converting to torch tensors ----
---- Dataset loaded! ----


In [65]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]
BBM_creator.instantiate_model(BBM2, input_size, output_size)

In [66]:
# Train the model
BBM_creator.train(save_to="models", epochs=500)

------ Training model 'BBM2-deep' ------
Models and summary will be save to 'models/' (will be created if it does not exist)
    - Model path: models\BBM2-deep_OPF-NI1-O3_20240201-112030.pt
    - Summary path: models/models_summary.csv
Training on cuda:0


Epoch 1/500 (Loss - Running: 177425.19, Avg. batch: 1.36e+02, Best avg.: N/A): 100%|██████████| 1304/1304 [00:05<00:00, 238.40it/s]
Epoch 2/500 (Loss - Running: 91348.16, Avg. batch: 7.03e+01, Best avg.: 69.84): 100%|██████████| 1304/1304 [00:05<00:00, 228.02it/s]
Epoch 3/500 (Loss - Running: 83909.76, Avg. batch: 6.45e+01, Best avg.: 69.84): 100%|██████████| 1304/1304 [00:05<00:00, 223.66it/s]
Epoch 4/500 (Loss - Running: 75796.01, Avg. batch: 5.83e+01, Best avg.: 69.84): 100%|██████████| 1304/1304 [00:05<00:00, 241.03it/s]
Epoch 5/500 (Loss - Running: 69199.15, Avg. batch: 5.32e+01, Best avg.: 54.34): 100%|██████████| 1304/1304 [00:05<00:00, 245.58it/s]
Epoch 6/500 (Loss - Running: 61228.89, Avg. batch: 4.71e+01, Best avg.: 48.33): 100%|██████████| 1304/1304 [00:05<00:00, 250.36it/s]
Epoch 7/500 (Loss - Running: 57830.33, Avg. batch: 4.45e+01, Best avg.: 47.53): 100%|██████████| 1304/1304 [00:05<00:00, 228.74it/s]
Epoch 8/500 (Loss - Running: 53982.57, Avg. batch: 4.15e+01, Best avg.

In [70]:
BBM_creator._summary()

## Raw dataset

In [68]:
# Set up the dataset
dataset_name = "OPF-I1-O3"
loaders = creator.setup_datasets(datasets_folder, dataset_name, remove_nans=True, ratios=(0.70, 0.15, 0.15), batch_size=32)
BBM_creator.set_dataloaders(*loaders)

In [69]:
# Instantiate model
input_size = loaders[0].dataset[0][0].shape[0]
output_size = loaders[0].dataset[0][1].shape[0]
BBM_creator.instantiate_model(BBM2, input_size, output_size)

In [21]:
# Train the model
BBM_creator.train(save_to="models", epochs=1000)

Training model 'BBM2_deep'
The folder 'models/' will be created if it does not exist
Training on cuda:0


Epoch 1/1000 (Loss - Running: 143714.25, Avg. batch: 1.11e+02, Best avg.: N/A): 100%|██████████| 1304/1304 [00:05<00:00, 238.15it/s]
Epoch 2/1000 (Loss - Running: 105456.75, Avg. batch: 8.11e+01, Best avg.: 88.77): 100%|██████████| 1304/1304 [00:05<00:00, 236.75it/s]
Epoch 3/1000 (Loss - Running: 102244.17, Avg. batch: 7.86e+01, Best avg.: 85.87): 100%|██████████| 1304/1304 [00:05<00:00, 238.58it/s]
Epoch 4/1000 (Loss - Running: 97576.24, Avg. batch: 7.51e+01, Best avg.: 80.03): 100%|██████████| 1304/1304 [00:05<00:00, 235.61it/s]
Epoch 5/1000 (Loss - Running: 95546.73, Avg. batch: 7.35e+01, Best avg.: 76.99): 100%|██████████| 1304/1304 [00:05<00:00, 235.18it/s]
Epoch 6/1000 (Loss - Running: 85302.18, Avg. batch: 6.56e+01, Best avg.: 73.84): 100%|██████████| 1304/1304 [00:05<00:00, 233.66it/s]
Epoch 7/1000 (Loss - Running: 72473.28, Avg. batch: 5.57e+01, Best avg.: 59.65): 100%|██████████| 1304/1304 [00:05<00:00, 217.44it/s]
Epoch 8/1000 (Loss - Running: 67377.93, Avg. batch: 5.18e+01,

In [22]:
BBM_creator._summary()